In [1]:
import mfapy
%matplotlib inline
import numpy as np
import scipy.integrate
from matplotlib import pyplot as plt

Outline for Master Code:
- Introduction markdown cell -> introduce code and what exactly it does
- Try some fancy markdown figures for model/model description
- Input cells -> have user input raw data (MDV files)
- Code to set up model and make necessary variables
- Code to run mfapy and display flux data
- Output in some graphs/tables
- Add extra spice as needed :)

In [2]:
#Input Checklist:
# - Simulation model .txt file --> defines reactions and metabolites
# - Simulation .csv file --> code calls values from the cells of this file
# - MDV data .txt file --> code calls values from this file to compute carbon fluxes

# User input code goes here:

simulation_model = input("Enter the name of the metabolic model .txt file: ")

simulation_csv = input("Enter the name of the .csv file containing the parameters: ")

MDV_data = input("Enter the name of the MDV .txt file: ")

Enter the name of the metabolic model .txt file:  Elim2_model.txt
Enter the name of the .csv file containing the parameters:  Elim2_status.csv
Enter the name of the MDV .txt file:  Elim2_mdv.txt


In [ ]:
print(MDV_data)

In [57]:
#Dumping some set up code here from Explanation 1:

#sets up model variables using simulation model.txt file
reactions, reversible, metabolites, target_fragments = mfapy.mfapyio.load_metabolic_model(simulation_model, format="text")
model = mfapy.metabolicmodel.MetabolicModel(reactions, reversible, metabolites, target_fragments)

#configurations -> investigate what these actually do and if we need to change them
#model.set_configuration(callbacklevel = 1) # Frequency level of callbacks from metabolic model
#model.set_configuration(iteration_max = 10000) # Maximal number of interations (steps) in each fitting task
#model.set_configuration(initial_search_repeats_in_grid_search = 5) # Number of initial metabolic flux disributions generated for each trial in grid search.
#model.set_configuration(initial_search_iteration_max = 1000) # Maximal number of interations (steps) allowed in each task to find feasible initial metabolic flux distribution.
#model.set_configuration(grid_search_iterations = 1) # Number of trials for model fitting at each point in grid search.
#model.set_configuration(number_of_repeat = 3) # Iteration in self.fitting_flux(method = 'deep') [SLSQP => LN_PRAXIS] * n
#model.set_configuration(ncpus = 4) # Number of CPUs for Parallel processing

#loads states and values from .csv file
flux = model.load_states(simulation_csv, format = 'csv')
model.set_constraints_from_state_dict(flux)
model.update()

Caution: Formate was not used in the metabolid network


True

In [ ]:
# Generation of CarbonSource instance
cs1 = model.generate_carbon_source_template()
cs1.set_each_isotopomer('CO2', {'#1':1.0})
#cs2 = model.generate_carbon_source_template()
#cs2.set_each_isotopomer('AcCoA', {'#11':0.5})
# What, if any, isotopomers do we need to set?

In [65]:
 # Isotope labelling of carbon sources from mfapy Ex#2
    #
carbon_source1 = model.generate_carbon_source_templete()
#carbon_source1.set_each_isotopomer('SubsGlc',{'#000000': 0.02, '#100000': 0.7, '#111111': 0.28 }, 
carbon_source1.set_each_isotopomer('SubsCO2',{'#0': 0.01, '#1': 0.99}, correction = 'yes') #correction = 'yes')
carbon_source1.set_each_isotopomer('Subsmeoh',{'#0': 1.0, '#1': 0.0}, correction = 'yes') #correction = 'yes')
#

True

In [66]:
# Load MDV data
mdv1 = model.load_mdv_data(MDV_data) #string variable from user input

#include more if there are more data files

In [67]:
# Flux estimation Step 1: Setting experimments
model.set_experiment('ex1', mdv1, carbon_source1)
#include more for more data sets

True

In [68]:
# Flux estimation step 2: Generation of intical flux vectors
endstate, flux = model.generate_initial_states(50, 4, method ="normal") #method ="parallel")


In [ ]:
model.generate_initial_states?

In [69]:
# Flux estimation step 3: Fitting model
for method in ["GN_CRS2_LM", "LN_PRAXIS", "SLSQP"]:
    endstate, RSS, flux = model.fitting_flux(method = method, flux = flux)
model.show_results([("final", flux[0])]) # Show result

Id             Reaction                 External i   final
RSS                                               238213.45
Thres                                                38.89
p_value                                               0.00
Id             Reaction                 External i   final Type     Value Stdev    lb    ub Atom_mapping             
r1             SubsGlc-->G6P            (kegg:R028     0.0 fixed      0.0   1.0   0.01000.0 ABCDEF-->ABCDEF          
r2             G6P-->F6P                (kegg:R007    39.1 free       1.0   1.0   0.01000.0 ABCDEF-->ABCDEF          
r3             F6P-->G6P                (kegg:R007    59.7 free       1.0   1.0   0.01000.0 ABCDEF-->ABCDEF          
r4             F6P-->FBP                (kegg:R007    90.2 free       1.0   1.0   0.01000.0 ABCDEF-->ABCDEF          
r5             FBP-->F6P                (kegg:R010   114.5 free       1.0   1.0   0.01000.0 ABCDEF-->ABCDEF          
r6             FBP-->DHAP+GAP           (kegg:R010    1

In [ ]:
# Estimation of 95% CI
ci_edge = model.generate_ci_template(targets = [('reaction', "v3")])
ci = model.search_ci(ci_edge, flux[0], method = 'grid')
lb = ci['data'][('reaction', "v3")]['lower_boundary']
ub = ci['data'][('reaction', "v3")]['upper_boundary']
print("v3", "Lower bondary:",lb, "Upper boundary:", ub)

In [ ]:
#Run mfapy code here:

In [ ]:
#Output data code goes here:

  # Output fitted metabolic flux into CSV file

output_file = input("Enter desired name of output .csv file: ")
    
model.show_results(results, filename = output_file,  checkrss = "on", format = "csv")
    
    # Output fitted metabolic flux in console
    
model.show_results(results,  checkrss = "on")
    
    # Output flux balance of given metabolite
    
model.show_flux_balance(results, "Fum")
model.show_flux_balance(results, "Fum", filename = "Explanation_1_13CMFA_metabolite_balance.csv", format = "csv")